<a href="https://colab.research.google.com/github/thejunemist/CNN/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'cnn/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
  !curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [7]:
USE_CUDA = torch.cuda.is_available()


def train_batch(_model, _x_tensor, _y_tensor, _target):
    _model.zero_grad()
    # input should be batch, alphabet, hidden_size
    out = _model(_x_tensor)
    _loss = _target(out.cuda(), _y_tensor.long())
    _loss.backward()
    for p in _model.parameters():
        if p.grad is not None:
            p.data.add_(-LEARNING_RATE, p.grad.data)
    torch.cuda.empty_cache()
    return out, _loss.item()


def evaluate(_model, _x_val, _y_val):
    out = _model(_x_val)
    out_proba, out_class = torch.max(out, 1)
    acc = accuracy_score(_y_val.type(torch.int32).numpy(),
                         out_class.cpu().type(torch.int32).numpy())
    f1 = f1_score(_y_val.type(torch.int32).numpy(),
                  out_class.cpu().type(torch.int32).numpy(), average='weighted')
    cr = classification_report(_y_val.type(torch.int32).numpy(),
                               out_class.cpu().type(torch.int32).numpy())
    return acc, f1, cr
Collapse

NameError: ignored

In [0]:
criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=REG_PARAM) #, momentum=MOMENTUM)
    #scheduler = CyclicLR(optimizer)
    stopper = EarlyStopping(patience=101, verbose=True, saver=False)
    for epoch in range(EPOCHS):
        el = 0
        model.train()
        for batch in train_batched:
            #scheduler.batch_step()
            x = Variable(batch[0].cuda(),  requires_grad=True)
            y = Variable(batch[1].cuda())
            preds, loss = train_batch(model, x, y, criterion)
            el += loss

        model.eval()
        for val_batch in valid_set:
            x_val = Variable(val_batch[0].cuda())
            y_val = Variable(val_batch[1].cuda())
            val_out = model(x_val)
            val_loss = criterion(val_out.cuda(), y_val.long())
        stopper(val_loss, model)

        if stopper.early_stop:
            print("Stopping training at epoch {cur}".format(cur=epoch))
            for test_batch in test_set:
                x_test = Variable(test_batch[0].cuda())
                y_test = Variable(test_batch[1])
                acc, f1, cr = evaluate(model, x_test, y_test)
                print('Test set accuracy of {a} and F1 of {f}'.format(a=acc, f=f1))
                print(cr)
            break

        if epoch % 5 == 0:
            for test_batch in test_set:
                x_test = Variable(test_batch[0].cuda())
                y_test = Variable(test_batch[1])
                acc, f1, cr = evaluate(model, x_test, y_test)
                print('Test set accuracy of {a} and F1 of {f}'.format(a=acc, f=f1))
                print(cr)
        print('Epoch {e} had loss {ls}'.format(e=epoch, ls=el))
        #lr_list = scheduler.get_lr()
    torch.save(model.state_dict(), 'outputs/conv_model.pt')
    #print('Final learning rate was {x}'.format(x=lr_list[-1]))